In [ ]:
import requests
import re
import tabula
import pandas as pd

# Download of Module Handbooks

In [ ]:
pdf_url_bachelor_business_management_and_economics = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-184-H-2008.pdf"
pdf_url_bachelor_information_systems = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-277-H-2021.pdf"
pdf_url_bachelor_economathematics = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-82-276-H-2015.pdf"
pdf_url_master_information_systems = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-88-j10-H-2019.pdf"
pdf_url_master_management ="https://www2.uni-wuerzburg.de/mhb/MHB1-en-88-i45-H-2018.pdf"
pdf_url_master_international_economic_policy = "https://www2.uni-wuerzburg.de/mhb/MHB1-en-88-h92-H-2018.pdf"

response = requests.get(pdf_url_bachelor_business_management_and_economics)
with open("module_handbooks/bachelor_business_management_and_economics.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded bachelor_business_management_and_economics.pdf")

response = requests.get(pdf_url_bachelor_information_systems)
with open("module_handbooks/bachelor_information_systems.pdf", "wb") as f:
    f.write(response.content)

print("Downloaded bachelor_information_systems.pdf")
response = requests.get(pdf_url_bachelor_economathematics)
with open("module_handbooks/bachelor_economathematics.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded bachelor_economathematics.pdf")

response = requests.get(pdf_url_master_information_systems)
with open("module_handbooks/master_information_systems.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded master_information_systems.pdf")

response = requests.get(pdf_url_master_management)
with open("module_handbooks/master_management.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded master_management.pdf")

response = requests.get(pdf_url_master_international_economic_policy)
with open("module_handbooks/master_international_economic_policy.pdf", "wb") as f:
    f.write(response.content)
print("Downloaded master_international_economic_policy.pdf")

# Ansatz über TABULA --> Verforfen 
Reading Tables from Module Book for master_information_systems

In [ ]:
import pandas as pd
import tabula
import re

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:


tables_master_is = tabula.read_pdf("module_handbooks/master_information_systems.pdf", pages='10-30', multiple_tables=True)
print(len(tables_master_is))
print(tables_master_is)

# PDF Miner Ansatz wie Felix -> über Regex

In [ ]:
import io
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfinterp import PDFPageInterpreter, PDFResourceManager
from pdfminer.pdfpage import PDFPage


def pdf_to_text(file_path, n=0):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    laparams = LAParams()

    laparams = LAParams()
    laparams.line_margin = 2.1
    laparams.word_margin = 0.1
    laparams.boxes_flow = 0.01
    # laparams.detect_vertical = True
    # laparams.all_texts = True


    converter = TextConverter(
        resource_manager,
        fake_file_handle,
        codec="utf-8",
        laparams=laparams,
    )
    page_interpreter = PDFPageInterpreter(resource_manager, converter)

    with open(file_path, "rb") as fh:
        for i, page in enumerate(PDFPage.get_pages(fh, caching=True, check_extractable=True)):
            if i < n:
                continue
            page_interpreter.process_page(page)

        text = fake_file_handle.getvalue()

    converter.close()
    fake_file_handle.close()

    if text:
        return remove_header_footer(text)
    else:
        return ""


def remove_header_footer(text):
    # Split the text into lines
    lines = text.split("\n")

    # Remove lines that match the header pattern
    lines = [line for line in lines if not line.startswith("JMU Würzburg")]

    # Remove lines that match the footer pattern
    lines = [line for line in lines if not line.startswith("page")]
    lines = [line for line in lines if not re.match(r".*Master.* with 1 major Information Systems.*", line)]
    lines = [line for line in lines if not re.match(r".*Master.* Information Systems - 2019.*", line)]
    lines = [line for line in lines if not re.match(r".*Module Catalogue for the Subject.*", line)]
    lines = [line for line in lines if not re.match(r".*Information Systems.*", line)]
    lines = [line for line in lines if not re.match(r".*Master’s with 1 major, 120 ECTS credits.*", line)]


    # Join the remaining lines back into a single string
    clean_text = "\n".join(lines)

    return clean_text

In [ ]:
text = pdf_to_text("module_handbooks/master_information_systems.pdf", 9)

In [ ]:
import io
import PyPDF2

def pdf_to_text_pypdf(file_path):
    text = ""
    with open(file_path, 'rb') as f:
        pdf_reader = PyPDF2.PdfReader(f)  # Use PdfReader instead of PdfFileReader
        num_pages = len(pdf_reader.pages)
        for page in range(num_pages):
            page_obj = pdf_reader.pages[page]
            text += page_obj.extract_text()
    return text

In [ ]:
text = pdf_to_text_pypdf("module_handbooks/master_information_systems.pdf")

In [ ]:
print(text)

In [ ]:
text_no_footer_header = remove_header_footer(text)
print(text_no_footer_header)

with open("module_handbooks/master_information_systems_processed.txt", "w") as f:
    f.write(text_no_footer_header)

In [308]:
tables = re.split(r"(?=Module title)", text_no_footer_header)[1:]
print(len(tables))
print("---------------TABLE 0--------------------------")
print(tables[2])
# store to table.txt file 
with open("module_handbooks/master_information_systems_table_0.txt", "w") as f:
    f.write(tables[0])

with open("module_handbooks/master_information_systems_table_1.txt", "w") as f:
    f.write(tables[1])

136
---------------TABLE 0--------------------------
Module title
Project Seminar
Module coordinator
Holder of the Chair of Business Management and Business 
ECTS Method of grading
numerical grade
15
Module level
Duration
graduate
1 semester
Contents
Content:
In small project teams of 4 to 10 members, students will spend several months actively working on a specific and 
realistic problem with practical relevance. They will progress through several project stages including as-is analy-
sis, to-be conception and implementation of an IS solution. The project teams will be required to work indepen-
dently and will only receive advice and minor support from research assistants.
Reading:
will vary according to topic
Intended learning outcomes
After completing the course "Projektseminar", students will be able to
1. analyze business tasks and requirements and generate fitting IS solutions;
2. apply project management methods;
3. internalize stress, time and conflict management by means of pr

In [310]:
def extract_table_values(table_string):
    patterns = {
        "Module title": r"Module title(.*?)\sModule coordinator",
        "Abbreviation": r"Abbreviation(.*?)\s+Module offered by",
        "Method of grading": r"grading(.*?)\s\d",
        "ECTS": r"numerical grade*(.*?)\sModule level",
        "Contents": r"Contents\s+(.*?)Intended learning outcomes",
        "Intended learning outcomes": r"Intended learning outcomes\s+(.*?)\s+Courses",
        "Methods of assessment": "Method of assessment  \(type, scope, language — if other than German, examination offered — if not every semester, information on whether module is creditable for bonus\)\s+(.*?)\s+Allocation",
        "Workload": r"Workload\s+(.*?)\s+Teaching cycle",
        "Teaching cycle": r"Teaching cycle\s+(.*?)\s+Referred to in LPO I",
        "Referred to in LPO I": r"Referred to in LPO I  \(examination regulations for teaching-degree programmes\)+(.*?)Module appears in",
        "Courses": r"German\)(.*?)Method",
        "Module appears in": r"Module appears in\s(.*?)\sAbbreviation",
        "Addititional information": r"Additional information\s+(.*?)\s+Workload",
        "Allocation of places": r"Allocation of places\s+(.*?)\s+Additional information",
        "Only after succ. compl. of module(s)":r"(?<=ECTS\s)([^\n]+)\s+Other prerequisites",
        "Module coordinator": r"Module coordinator(.*?)\s+ECTS",
        # "Duration": r"Duration\s+(\d\ssemester|1\ssemester)\s",
        "Module offered by": r"Module offered by(.*?)Only after",
    }   

    extracted_values = {}

    # convert the pattern.items() to lowercase
    patterns = {key.lower(): value for key, value in patterns.items()}

    # Extract values using regex
    for key, pattern in patterns.items():
        match = re.search(pattern, table_string, re.DOTALL)
        if match:
            extracted_values[key] = match.group(1).strip()

    return extracted_values

In [311]:
print("---------------TABLE 0--------------------------")
# print(tables[0])
values = extract_table_values(tables[0])
# Print the extracted values
for key, value in values.items():
    print(key + ": " + value)


print("\n---------------TABLE 1--------------------------")
values = extract_table_values(tables[1])
for key, value in values.items():
    print(key + ": " + value)

print("\n---------------TABLE 2--------------------------")
values = extract_table_values(tables[2])
for key, value in values.items():
    print(key + ": " + value)

print("\n---------------TABLE 3--------------------------")
values = extract_table_values(tables[3])
for key, value in values.items():
    print(key + ": " + value)


---------------TABLE 0--------------------------
module title: Information Processing within Organizations
abbreviation: 12-M-ITM-161-m01
method of grading: numerical grade
ects: 5
contents: Content:
This course provides students with an in-depth overview of the structure and the application areas of business 
management information systems in enterprises and public institutions.
Outline of syllabus:
1. What is software: concepts, categories, application
2. Software life cycle: duration, phases, steps
3. As-is analysis: tasks, problems
4. To-be concept: system design, data design, dialog design, function design
5. Object orientation: paradigm shift
6. Change management: meaning, methodologies, project management
7. Office automation: tasks, areas of application
intended learning outcomes: After completing the course "Integrated Information Processing", students will be able to
(i) understand the importance of integration in enterprises, especially in information systems;
(ii) assess th

# Create final Dataframe with these columns

In [ ]:
import pandas as pd

In [ ]:
module_handbook_columns = [
    "Module title",
    "Abbreviation",
    "Module coordinator",
    "Module offered by",
    "ECTS",
    "Method of grading",
    "Completion of Modules",
    "Duration",
    "Module Level",
    "Other Prerequisites",
    "Contents",
    "Intended Learning Outcomes",
    "Courses",
    "Method of Assessment",
    "Allocation of Places",
    "Additional Information",
    "Workload",
    "Teaching Cycle",
    "Referred to in LPO I",
    "Module appears in"
]


In [305]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


def process_module_handbook(tables):
    module_handbook_columns = [
        "Module title",
        "Abbreviation",
        "Module coordinator",
        "Module offered by",
        "ECTS",
        "Method of grading",
        "Completion of Modules",
        "Duration",
        "Module Level",
        "Other Prerequisites",
        "Contents",
        "Intended Learning Outcomes",
        "Courses",
        "Method of Assessment",
        "Allocation of Places",
        "Additional Information",
        "Workload",
        "Teaching Cycle",
        "Referred to in LPO I",
        "Module appears in",
    ]
    # convert to lowercase
    module_handbook_columns = [column.lower() for column in module_handbook_columns]
    module_handbook_df = pd.DataFrame(columns=module_handbook_columns)

    for table in tables:
        values = extract_table_values(table)
        # Append the values as a new row to the module_handbook_df dataframe
        module_handbook_df = module_handbook_df.append(values, ignore_index=True)

    return module_handbook_df


df_compl = process_module_handbook(tables)
# store to csv file and excel
df_compl.to_csv("module_handbooks/master_information_systems.csv", index=False)


In [312]:
print(df_compl.head())
print(df_compl.shape)
print(df_compl.iloc[0])

                                  module title      abbreviation  \
0  Information Processing within Organizations  12-M-ITM-161-m01   
1                                IT-Management     12-PS-192-m01   
2                              Project Seminar   10-I=IR-161-m01   
3                        Information Retrieval   10-I=PA-161-m01   
4              Analysis and Design of Programs  10-I=SSS-172-m01   

                                  module coordinator  \
0  holder of the Chair of Business Management and...   
1                                                      
2  Holder of the Chair of Business Management and...   
3      Dean of Studies Informatik (Computer Science)   
4         holder of the Chair of Computer Science II   

                              module offered by ects method of grading  \
0  Faculty of Business Management and Economics    5   numerical grade   
1  Faculty of Business Management and Economics    5   numerical grade   
2                 Institute of C

In [313]:
def process_module_handbook(file_path):
    text = pdf_to_text(file_path, n=9)
    print("Done with converting to string")

    text_no_footer_header = remove_header_footer(text)
    print("Done with removing header and footer")

    tables = re.split(r"(?=Module title)", text_no_footer_header)[1:]
    print("Done with splitting into tables")

    module_handbook_columns = [
        "Module title",
        "Abbreviation",
        "Module coordinator",
        "Module offered by",
        "ECTS",
        "Method of grading",
        "Completion of Modules",
        "Duration",
        "Module Level",
        "Other Prerequisites",
        "Contents",
        "Intended Learning Outcomes",
        "Courses",
        "Method of Assessment",
        "Allocation of Places",
        "Additional Information",
        "Workload",
        "Teaching Cycle",
        "Referred to in LPO I",
        "Module appears in",
    ]
    # convert to lowercase
    module_handbook_columns = [column.lower() for column in module_handbook_columns]
    module_handbook_df = pd.DataFrame(columns=module_handbook_columns)

    for table in tables:
        values = extract_table_values(table)
        # print("Done with extracting values for table")
        # Append the values as a new row to the module_handbook_df dataframe
        module_handbook_df = module_handbook_df.append(values, ignore_index=True)

    return module_handbook_df

df_compl = process_module_handbook("module_handbooks/master_information_systems.pdf")

Done with converting to string
Done with removing header and footer
Done with splitting into tables


In [317]:
print(df_compl.shape)
print(df_compl.head(20))

(136, 21)
                                      module title        abbreviation  \
0      Information Processing within Organizations    12-M-ITM-161-m01   
1                                    IT-Management       12-PS-192-m01   
2                                  Project Seminar     10-I=IR-161-m01   
3                            Information Retrieval     10-I=PA-161-m01   
4                  Analysis and Design of Programs    10-I=SSS-172-m01   
5                     Security of Software Systems    10-I=SAR-161-m01   
6                            Software Architecture    10-I=KI1-161-m01   
7                        Artificial Intelligence 1     10-I=ST-161-m01   
8                        Discrete Event Simulation    10-I=APR-182-m01   
9                             Advanced Programming    10-I=PNN-212-m01   
10                    Programming with neural nets    10-I=STM-162-m01   
11                             NLP and Text Mining     10-I=SB-212-m01   
12                          

In [ ]:
# apply the function to the module_handbook_df

module_handbook_columns = [
    "Module title",
    "Abbreviation",
    "Module coordinator",
    "Module offered by",
    "ECTS",
    "Method of grading",
    "Only after successfull completion of module(s)",
    "duration",
    "module level",
    "other prerequisites",
    "contents",
    "intended learning outcomes",
    "courses",
    "method of assessment",
    "Allocation of places",
    "Additional information",
    "Workload",
    "Teaching cycle",
    "Referred to in LPO I", "page"]
module_handbook_df = pd.DataFrame(columns=module_handbook_columns)
module_handbook_df.to_csv(
    "module_handbooks/master_information_systems.csv", index=False
)


module_handbook_df = process_module_handbook("module_handbooks/master_information_systems.pdf", module_handbook_df)